In [ ]:
import pandas as pd
import requests

# Set the parameters for the API request
params = {
    "dataset": "daily-summaries",
    "stations": "GHCND:USW00094728", # This is the station ID for Central Park, NY
    "startdate": "1921-01-01",
    "enddate": "2020-12-31",
    "datatypeid": "TAVG", # This is the ID for average temperature
    "limit": 100000, # This is the maximum number of records we can get in a single request
    "units": "metric",
    "includemetadata": "false",
}

# Make the API request
response = requests.get("https://www.ncei.noaa.gov/access/services/data/v1", params=params)

# Convert the response to a Pandas dataframe
data = pd.read_csv(response.content.decode("utf-8"))

# Convert the date column to a datetime object
data["DATE"] = pd.to_datetime(data["DATE"])

# Group the data by year and month, and calculate the average temperature for each month
monthly_data = data.groupby([data["DATE"].dt.year, data["DATE"].dt.month])["TAVG"].mean()

# Print the resulting dataframe
print(monthly_data)


In [ ]:
import pandas as pd
import requests

# Set the parameters for the API request
params = {
    "dataset": "daily-summaries",
    "stations": "GHCND:KPC00094641", # This is the station ID for Seoul, South Korea
    "startdate": "1921-01-01",
    "enddate": "2020-12-31",
    "datatypeid": "TAVG", # This is the ID for average temperature
    "limit": 100000, # This is the maximum number of records we can get in a single request
    "units": "metric",
    "includemetadata": "false",
}

# Make the API request
response = requests.get("https://www.ncei.noaa.gov/access/services/data/v1", params=params)

# Convert the response to a Pandas dataframe
data = pd.read_csv(response.content.decode("utf-8"))

# Convert the date column to a datetime object
data["DATE"] = pd.to_datetime(data["DATE"])

# Group the data by year and month, and calculate the average temperature for each month
monthly_data = data.groupby([data["DATE"].dt.year, data["DATE"].dt.month])["TAVG"].mean()

# Print the resulting dataframe
print(monthly_data)


## from landsat

In [2]:
import ee
# ee.Authenticate() # follow the authentication steps in the opened web page
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=IA9lpwHOoyOdZX2CpY4Jg8fybYKD40-aFxgCg9V7ltg&tc=F5HcYIltXGD3gxyrOBwRkjTig123aNRdMqZPfgZECEY&cc=K0CXA93tr8j6qe6y8opcWPMTPfv732IeYR3r7CbeBNg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh75Rxa7RwsCUuW3XAB5Eir0ixtbsOsKkktfsP0v6M7tsCpKd9RQhZk

Successfully saved authorization token.


In [ ]:


# Define the region of interest (ROI)
roi = ee.Geometry.Rectangle([126.63, 37.37, 127.03, 37.63]) # [xmin, ymin, xmax, ymax] in decimal degrees

# Define the time period
start_date = '2019-01-01'
end_date = '2019-12-31'

# Load the Landsat 8 surface reflectance data
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
  .filterBounds(roi) \
  .filterDate(start_date, end_date)

# Select the bands of interest
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# Get the first image in the collection
image = collection.first()

# Get the metadata of the image
metadata = image.getInfo()

# Get the image ID and date
image_id = metadata['id']
image_date = metadata['properties']['DATE_ACQUIRED']

# Download the image
download_url = image.getDownloadURL({
    'name': image_id,
    'bands': ','.join(bands),
    'scale': 30 # the resolution of the image in meters
})

# Download the image to a local file
import urllib.request
urllib.request.urlretrieve(download_url, f'{image_id}.zip')


In [6]:
metadata['properties']

{'IMAGE_QUALITY_TIRS': 9,
 'CLOUD_COVER': 68.01,
 'EARTH_SUN_DISTANCE': 0.983303,
 'LANDSAT_ID': 'LC08_L1TP_116034_20190104_20190130_01_T1',
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[126.02314315859896, 38.4957677946388],
   [125.87395546104199, 38.522477152479325],
   [125.86981179067439, 38.5086024457793],
   [125.85873066100417, 38.470736942637245],
   [125.61875566020609, 37.636209918423724],
   [125.50183825328823, 37.2234571480061],
   [125.38369216827634, 36.802284501499194],
   [125.3865079570094, 36.80162373701499],
   [125.67434014572358, 36.74871365129544],
   [127.04364569637272, 36.48628387778768],
   [127.40986626337289, 36.413087333535465],
   [127.5111610592346, 36.74215116149085],
   [127.63079135184466, 37.127712459773484],
   [127.67904722490215, 37.282268829075186],
   [127.81183186042422, 37.70534392268486],
   [127.89883357477783, 37.98028984778154],
   [127.93717602575434, 38.10129561479134],
   [127.94694097064483, 38.132211316840916],
   [1

## next trial 

In [ ]:
!pip install overpy

In [9]:
import overpy

# Set up the Overpass API client
api = overpy.Overpass()

# Define the query to retrieve the administrative boundary data for South Korea
query = """
area["ISO3166-1"="KR"][admin_level=2];
relation(area)["type"="boundary"]["boundary"="administrative"]["admin_level"="3"];
out geom;
"""

# Send the query to the Overpass API
result = api.query(query)

# Print the result
for boundary in result.relations:
    print(boundary.tags['name'], boundary.geometry())


In [ ]:
import requests

# Set the URL for the Natural Earth data
url = 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries_lakes.zip'

# Download the data and extract the GeoJSON file for South Korea
import zipfile
import io
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
filename = 'ne_10m_admin_0_countries_lakes/ne_10m_admin_0_countries_lakes.shp'
gdf = gpd.read_file(z.open(filename))
gdf = gdf[gdf['ADMIN'] == 'South Korea']

# Print the resulting GeoDataFrame
print(gdf)


### yearly temp data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the URL for the GHCN data
url = 'https://www.ncei.noaa.gov/pub/data/ghcn/daily/hcn/'
filename = 'ALL_{}_{}_{}_dly.csv'

# Define the station ID for South Korea (in this case, Seoul)
station_id = 'KXSG0007'

# Define the start and end years for the temperature data
start_year = 1921
end_year = 2020

# Create an empty list to store the annual mean temperature data
annual_temps = []

# Loop over the years and retrieve the daily temperature data
for year in range(start_year, end_year + 1):
    df = pd.read_csv(url + filename.format(station_id, year, year), header=None)
    df.columns = ['station', 'date', 'element', 'value', 'mflag', 'qflag', 'sflag', 'obstime']
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
    df = df.set_index('date')
    df = df[df['element'] == 'TAVG']
    annual_temps.append(df.resample('Y').mean()['value'].values[0])

# Create a pandas Series with the annual mean temperature data
annual_temps_series = pd.Series(annual_temps, index=range(start_year, end_year + 1))

# Create a line plot of the annual mean temperature data
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(annual_temps_series.index, annual_temps_series, color='red')
ax.set_xlabel('Year')
ax.set_ylabel('Temperature (°C)')
ax.set_title('South Korea Annual Mean Temperature (1921-2020)')
plt.show()


### monthly data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the URL for the GHCN data
url = 'https://www.ncei.noaa.gov/pub/data/ghcn/daily/hcn/'
filename = 'ALL_{}_{}_{}_dly.csv'

# Define the station ID for South Korea (in this case, Seoul)
# station_id = 'KXSG0007'
station_id ='KSXX3778'

# Define the start and end years for the temperature data
start_year = 1921
end_year = 2020

# Create an empty list to store the monthly mean temperature data
monthly_temps = []

# Loop over the years and retrieve the daily temperature data
for year in range(start_year, end_year + 1):
    df = pd.read_csv(url + filename.format(station_id, year, year), header=None)
    df.columns = ['station', 'date', 'element', 'value', 'mflag', 'qflag', 'sflag', 'obstime']
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
    df = df.set_index('date')
    df = df[df['element'] == 'TAVG']
    monthly_temps.append(df.resample('M').mean()['value'].values)

# Create a pandas DataFrame with the monthly mean temperature data
monthly_temps_df = pd.DataFrame(monthly_temps, index=range(start_year, end_year + 1), columns=range(1, 13))

# Create a line plot of the monthly mean temperature data
fig, ax = plt.subplots(figsize=(12, 6))
monthly_temps_df.mean().plot(ax=ax, color='red')
ax.set_xlabel('Month')
ax.set_ylabel('Temperature (°C)')
ax.set_title('South Korea Monthly Mean Temperature (1921-2020)')
plt.show()


### Landsat temperature data with bounding box 

Landsat data does not provide temperature data for a period of 100 years. Landsat satellites capture multispectral imagery of the earth's surface, which can be used to derive various geospatial products, such as land cover, vegetation indices, and surface temperature. However, the Landsat program only began in the 1970s, so it does not cover the entire 20th century.

In [ ]:
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt

# Define the bounding box coordinates in WGS84 (longitude, latitude)
bbox = (126.813064, 37.447941, 129.468416, 38.676947)  # example for South Korea

# Read the temperature data from a file (assuming it is in GeoTIFF format)
with rasterio.open('path/to/temperature_data.tif') as src:
    temp_data, temp_transform = rasterio.mask.mask(src, [bbox], crop=True)

# Read the Landsat imagery from a file (assuming it is in GeoTIFF format)
with rasterio.open('path/to/landsat_data.tif') as src:
    landsat_data, landsat_transform = rasterio.mask.mask(src, [bbox], crop=True)

# Create a GeoDataFrame with the bounding box coordinates
geometry = gpd.points_from_xy([bbox[0], bbox[2]], [bbox[1], bbox[3]])
bbox_df = gpd.GeoDataFrame(geometry=geometry, crs='EPSG:4326')

# Plot the temperature data and Landsat imagery over the bounding box
fig, ax = plt.subplots(figsize=(12, 6))
ax.imshow(temp_data[0], cmap='coolwarm')
ax.imshow(landsat_data[0], alpha=0.5)
bbox_df.plot(ax=ax, color='none', edgecolor='black')
ax.set_axis_off()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Download the GHCN data for South Korea from 1921 to 2021
url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/hcn/'
query = 'hcn_'
station = '159'
start_year = 1921
end_year = 2021

data = pd.DataFrame()

for year in range(start_year, end_year + 1):
    filename = query + station + '_' + str(year) + '.csv'
    filepath = url + str(year) + '/' + filename
    try:
        year_data = pd.read_csv(filepath, header=None, names=['ID', 'Date', 'Element', 'Value', 'MFLAG', 'QFLAG', 'SFLAG', 'OBSTIME'])
        year_data = year_data[year_data['Element'] == 'TAVG']
        year_data['Year'] = year
        data = pd.concat([data, year_data])
    except:
        pass

# Convert the temperature data from tenths of degrees Celsius to degrees Celsius
data['Value'] = data['Value'] / 10

# Group the data by year and month and compute the monthly average temperature
monthly_data = data.groupby(['Year', pd.Grouper(key='Date', freq='M')])['Value'].mean().reset_index()

# Plot the monthly average temperature data over time
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(monthly_data['Date'], monthly_data['Value'], color='red')
ax.set_xlabel('Year')
ax.set_ylabel('Monthly Average Temperature (°C)')
ax.set_title('South Korea Monthly Average Temperature (1921-2021)')
plt.show()


In [ ]:
# https://www.ncei.noaa.gov/access/search/data-search/daily-summaries?bbox=37.611,126.933,37.521,127.023&startDate=2018-12-03T00:00:00&endDate=2020-01-03T23:59:59

The GHCN-D dataset does not include geographic coordinates for the weather stations, so it's not possible to select data for a specific region using a bounding box. The dataset only includes the station ID, name, country, and location (e.g. city or airport).

If you want to select temperature data for a specific region in South Korea, you can try to find a weather station in that region using the dataset, and then select the data for that station as described in my earlier response. Alternatively, you can try to find a different dataset that includes geographic coordinates for weather stations in South Korea, and use that to select data for a specific region.

One such dataset that includes geographic coordinates is the Global Surface Summary of the Day (GSOD) dataset, which is also provided by the National Centers for Environmental Information (NCEI). The GSOD dataset is similar to the GHCN-D dataset, but includes additional variables and geographic coordinates for each weather station. You can follow the same steps as before to access the GSOD dataset from the NCEI website, but select "Global Surface Summary of the Day (GSOD)" instead of "Global Historical Climatology Network - Daily (GHCN-D)" in step 6.

Once you have downloaded the data and selected the data for the desired station and year range, you can plot the data as described in my earlier response. However, keep in mind that the GSOD dataset may have different data quality and processing procedures compared to the GHCN-D dataset, so you should check the data documentation and quality flags before using the data for any analysis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the URL for the GHCN-D dataset
url = "https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access/"

# Define the station ID for South Korea (Seoul)
station_id = "KSW00043242"

# Define the start and end years for the analysis
start_year = 1921
end_year = 2021

# Define a list to store the dataframes for each year
dfs = []

# Loop over each year and download the daily data for the station
for year in range(start_year, end_year+1):
    print(f"Downloading data for year {year}")
    file_url = url + f"{year}/{station_id}-{year}.csv"
    df = pd.read_csv(file_url, header=None, names=["station_id", "date", "element", "value", "m_flag", "q_flag", "s_flag", "obs_time"])
    df = df[df["element"] == "TAVG"]  # Keep only daily average temperature data
    df["date"] = pd.to_datetime(df["date"])  # Convert date to datetime object
    df = df.set_index("date")  # Set date as index
    dfs.append(df)

# Concatenate the dataframes for each year into a single dataframe
temp_data = pd.concat(dfs)

# Resample the data to compute the monthly average temperature
monthly_avg_temp_data = temp_data.resample("M").mean()

# Plot the monthly average temperature data over time
plt.figure(figsize=(12, 6))
plt.plot(monthly_avg_temp_data.index, monthly_avg_temp_data["value"], color="red")
plt.xlabel("Year")
plt.ylabel("Monthly Average Temperature (°C)")
plt.title("South Korea Monthly Average Temperature (1921-2021)")
plt.show()


### google earth engine code 

In [ ]:
var southKorea = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
  .filter(ee.Filter.eq('country_na', 'South Korea'));

var startDate = '1970-01-01';
var endDate = '2023-02-15';

var lst = ee.ImageCollection('LANDSAT/LC08/C01/T1_L2')
  .filterBounds(southKorea)
  .filterDate(startDate, endDate)
  .select('LST');


In [ ]:
Export.image.toDrive({
  image: lst.mean(),
  description: 'LST_SouthKorea',
  folder: 'GEE',
  region: southKorea.geometry().bounds(),
  scale: 30,
  maxPixels: 1e12
});


### Gee in python 

In [20]:
# !pip install earthengine-api


In [21]:
import ee
# ee.Authenticate()
ee.Initialize()

In [22]:
south_korea = ee.Geometry.Polygon(
    [[[123.588600, 33.204140], [131.081390, 33.204140], [131.081390, 38.612690], [123.588600, 38.612690], [123.588600, 33.204140]]], 
    None, 
    False)


In [23]:
start_date = '1970-01-01'
end_date = '2023-02-15'

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_L2') \
    .filterBounds(south_korea) \
    .filterDate(start_date, end_date) \
    .select('LST')


In [ ]:
task = ee.batch.Export.image.toDrive(
    image = collection.mean(),
    description = 'LST_SouthKorea',
    folder = 'GEE',
    region = south_korea.bounds().getInfo()['coordinates'],
    scale = 30,
    maxPixels = 1e12
)

task.start()


In [27]:
task.status()

{'state': 'UNSUBMITTED'}

In [ ]:
collection.size()

##easy code

In [41]:
import requests
import pandas as pd

def download_temperature_data(station_id, start_year, end_year):
    # Define the URL of the GHCN daily dataset
    url = "https://www.ncei.noaa.gov/access/services/data/v1"

    # Set the parameters for the data request
    params = {
        "dataset": "daily-summaries",
        "stations": station_id,
        "startDate": f"{start_year}-01-01",
        "endDate": f"{end_year}-12-31",
        "format": "json",
    }

    # Send the request to the server and get the response
    response = requests.get(url, params=params)
    print(response.code)

    # Parse the JSON response into a pandas DataFrame
    data = pd.read_json(response.content)

    # Extract the temperature data and return it
    temperature_data = data[["DATE", "TMAX", "TMIN"]].copy()
    temperature_data["DATE"] = pd.to_datetime(temperature_data["DATE"], format="%Y-%m-%d")
    temperature_data.set_index("DATE", inplace=True)

    return temperature_data


In [ ]:
temperature_data = download_temperature_data(station_id="KRS00047800", start_year=1970, end_year=2019)
print(temperature_data.head())


### Modis data 

In [44]:
import ee
import datetime

# Initialize the Earth Engine API
ee.Initialize()

# Define the bounding box of interest (longitude, latitude)
bbox = [127.0093, 37.5042, 127.1122, 37.5691]

# Define the time period of interest
start_date = datetime.datetime(1970, 1, 1)
end_date = datetime.datetime(2020, 12, 31)

# Load the MODIS Land Surface Temperature product
lst = ee.ImageCollection("MODIS/006/MOD11A2") \
        .filterBounds(ee.Geometry.Rectangle(bbox)) \
        .filterDate(start_date, end_date) \
        .select("LST_Day_1km", "LST_Night_1km")

# Get the temperature data as a time series
timeseries = lst.getRegion(ee.Geometry.Rectangle(bbox), scale=1000) \
        .getInfo()

# Convert the time series data to a Pandas DataFrame
import pandas as pd
data = pd.DataFrame(timeseries[1:], columns=timeseries[0])

# Convert the date column to a Python datetime object
data['time'] = pd.to_datetime(data['time'], unit='ms')

# Convert the temperature values from Kelvin to Celsius
data['LST_Day_1km'] = data['LST_Day_1km'].astype(float) - 273.15
data['LST_Night_1km'] = data['LST_Night_1km'].astype(float) - 273.15

# Print the first few rows of the data frame
print(data.head())


           id  longitude   latitude       time  LST_Day_1km  LST_Night_1km
0  2000_02_18  127.01729  37.509155 2000-02-18     13582.85            NaN
1  2000_02_26  127.01729  37.509155 2000-02-26     13846.85       13429.85
2  2000_03_05  127.01729  37.509155 2000-03-05     13924.85       13377.85
3  2000_03_13  127.01729  37.509155 2000-03-13     14077.85       13565.85
4  2000_03_21  127.01729  37.509155 2000-03-21     14157.85       13625.85


In [45]:
import ee
import pandas as pd

# Initialize the Earth Engine API
ee.Initialize()

# Define the bounding box of interest (longitude, latitude)
bbox = [127.0093, 37.5042, 127.1122, 37.5691]

# Load the MODIS Land Surface Temperature product
lst = ee.ImageCollection("MODIS/006/MOD11A2") \
        .filterBounds(ee.Geometry.Rectangle(bbox)) \
        .select("LST_Day_1km", "LST_Night_1km")

# Define the time period of interest
start_date = '2019-01-01'
end_date = '2019-12-31'

# Filter the image collection by date
lst_filtered = lst.filterDate(start_date, end_date)

# Get the temperature data as a time series
timeseries = lst_filtered.getRegion(
    ee.Geometry.Rectangle(bbox),
    scale=1000,
    crs='EPSG:4326'
  ).getInfo()

# Convert the time series data to a Pandas DataFrame
header = timeseries[0]
data = pd.DataFrame(timeseries[1:], columns=header)

# Convert the date column to a Python datetime object
data['time'] = pd.to_datetime(data['time'], unit='ms')

# Convert the temperature values from Kelvin to Celsius
data['LST_Day_1km'] = data['LST_Day_1km'].astype(float) - 273.15
data['LST_Night_1km'] = data['LST_Night_1km'].astype(float) - 273.15

# Print the first few rows of the data frame
print(data.head())


           id  longitude   latitude       time  LST_Day_1km  LST_Night_1km
0  2019_01_01  127.01729  37.509155 2019-01-01     13416.85       13189.85
1  2019_01_09  127.01729  37.509155 2019-01-09     13563.85       13314.85
2  2019_01_17  127.01729  37.509155 2019-01-17     13633.85       13331.85
3  2019_01_25  127.01729  37.509155 2019-01-25     13593.85       13247.85
4  2019_02_02  127.01729  37.509155 2019-02-02     13508.85       13116.85


In [ ]:
# import ee
# import pandas as pd

# # Initialize the Earth Engine API
# ee.Initialize()

# # Load the country boundary shapefile as a feature
# country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
#             .filter(ee.Filter.eq('country_na', 'South Korea'))

# # Load the MODIS Land Surface Temperature product
# lst = ee.ImageCollection("MODIS/006/MOD11A2") \
#         .filterBounds(country) \
#         .select("LST_Day_1km", "LST_Night_1km")

# # Define the time period of interest
# start_date = '2019-01-01'
# end_date = '2019-12-31'

# # Filter the image collection by date
# lst_filtered = lst.filterDate(start_date, end_date)

# # Get the temperature data as a time series
# timeseries = lst_filtered.getRegion(
#     country.geometry().bounds(),
#     scale=1000,
#     crs='EPSG:4326'
#   ).getInfo()

# # Convert the time series data to a Pandas DataFrame
# header = timeseries[0]
# data = pd.DataFrame(timeseries[1:], columns=header)

# # Convert the date column to a Python datetime object
# data['time'] = pd.to_datetime(data['time'], unit='ms')

# # Convert the temperature values from Kelvin to Celsius
# data['LST_Day_1km'] = data['LST_Day_1km'].astype(float) - 273.15
# data['LST_Night_1km'] = data['LST_Night_1km'].astype(float) - 273.15

# # Print the first few rows of the data frame
# print(data.head())


In [52]:
# ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME').filter(ee.Filter.eq('ADM0_NAME', "South Korea"))

In [ ]:
# ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Republic of Korea')).first()).geometry()

In [125]:
aa=country_boundary.bounds().getInfo()
aa['geometry']['coordinates']

[[[124.60813861452617, 33.106109727262016],
  [130.92321566233534, 33.106109727262016],
  [130.92321566233534, 38.614950058033024],
  [124.60813861452617, 38.614950058033024],
  [124.60813861452617, 33.106109727262016]]]

In [ ]:
import ee
import pandas as pd

# Initialize the Earth Engine API
ee.Initialize()

# Load the South Korea boundary shapefile as a feature collection
# country_boundary_fc = ee.FeatureCollection("users/your_username/your_feature_collection_name")
country_boundary=ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Republic of Korea')).first())
# # Load the South Korea boundary feature
# country_boundary = country_boundary_fc.first()

# Load the MODIS Land Surface Temperature product
lst = ee.ImageCollection("MODIS/006/MOD11A2") \
        .filterBounds(country_boundary) \
        .select("LST_Day_1km", "LST_Night_1km")

# Define the time period of interest
start_date = '2019-01-01'
end_date = '2019-12-31'

# Filter the image collection by date
lst_filtered = lst.filterDate(start_date, end_date)

# Get the temperature data as a time series
timeseries = lst_filtered.getRegion(
    country_boundary.geometry().bounds(),
    scale=1000,
    crs='EPSG:4326'
  ).getInfo()

# Convert the time series data to a Pandas DataFrame
header = timeseries[0]
data = pd.DataFrame(timeseries[1:], columns=header)

# Convert the date column to a Python datetime object
data['time'] = pd.to_datetime(data['time'], unit='ms')

# Convert the temperature values from Kelvin to Celsius
data['LST_Day_1km'] = data['LST_Day_1km'].astype(float) - 273.15
data['LST_Night_1km'] = data['LST_Night_1km'].astype(float) - 273.15

# Print the first few rows of the data frame
print(data.head())


In [86]:
map = folium.Map(location=[37, 126], zoom_start=14, control_scale=True)
map